## Forked from https://www.kaggle.com/code/abdurrafae/improved-code-interpretation

**Lewis:** the only changes in this notebook are those needed to run the original one with the new Kaggle evaluation API

In [1]:
import sys
import pkg_resources
import torch
import time
import aimo
import gc
import transformers
import re
import sys
import subprocess
import math
import random

import pandas as pd
import numpy as np

from numpy.random import choice
from collections import defaultdict, Counter
from tqdm import tqdm
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    AutoConfig,
    StoppingCriteria,
    set_seed,
    StoppingCriteriaList
)



/tmp/ipykernel_24/2184763029.py:2: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


In [2]:
DEBUG = False
QUANT = False
USE_PAST_KEY = True
VALIDATION = False
PRIVATE = True
EXTERNAL_DF = '/kaggle/input/ai-mathematical-olympiad-prize/train.csv'

if PRIVATE:
    MODEL_PATH = "/kaggle/input/deepseek-math"#"/kaggle/input/gemma/transformers/7b-it/1"
    DEEP = True

#n_repetitions = 17 if PRIVATE else 4 # Original notebook had 22 but times out :(
n_repetitions = 1
TOTAL_TOKENS = 2048 # if PRIVATE else 512

if PRIVATE:
    TIME_LIMIT = 31500
else:
    TIME_LIMIT = 1
    
aops = """Here are some example math problems along with explanations:

Q1. Cities $A$ and $B$ are $45$ miles apart. Alicia lives in $A$ and Beth lives in $B$. Alicia bikes towards $B$ at 18 miles per hour. Leaving at the same time, Beth bikes toward $A$ at 12 miles per hour. How many miles from City $A$ will they be when they meet?
A1. This is a $d=st$ problem, so let $x$ be the time it takes to meet. We can write the following equation:\[12x+18x=45\]Solving gives us $x=1.5$. The $18x$ is Alicia so so $18\times1.5=\boxed{27}$

Q2. How many positive perfect squares less than $2023$ are divisible by $5$?
A2. Since $\left \lfloor{\sqrt{2023}}\right \rfloor = 44$, there are $\left \lfloor{\frac{44}{5}}\right \rfloor = \boxed{8}$ perfect squares less than 2023.

Q3. How many digits are in the base-ten representation of $8^5 \cdot 5^{10} \cdot 15^5$?
A3. Prime factorizing this gives us $2^{15}\cdot3^{5}\cdot5^{15}=10^{15}\cdot3^5=243\cdot10^{15}$. $10^{15}$ has $16$ digits and $243$ = $2.43*10^{2}$ gives us $2$ more digits. $16+2=\text{\boxed{18}$. $2.43*10^{17}$ has $18$ digits

Q4. A digital display shows the current date as an $8$-digit integer consisting of a $4$-digit year, followed by a $2$-digit month, followed by a $2$-digit date within the month. For example, Arbor Day this year is displayed as 20230428. For how many dates in $2023$ will each digit appear an even number of times in the 8-digital display for that date?
A4. Do careful casework by each month. In the month and the date, we need a $0$, a $3$, and two digits repeated (which has to be $1$ and $2$ after consideration). After the casework, we get $\boxed{9}$.

Q5. Maureen is keeping track of the mean of her quiz scores this semester. If Maureen scores an $11$ on the next quiz, her mean will increase by $1$. If she scores an $11$ on each of the next three quizzes, her mean will increase by $2$. What is the mean of her quiz scores currently?
A5. Let $a$ represent the amount of tests taken previously and $x$ the mean of the scores taken previously. We can write the following equations:

\[\frac{ax+11}{a+1}=x+1\qquad (1)\]\[\frac{ax+33}{a+3}=x+2\qquad (2)\]

Multiplying $(x+1)$ by $(a+1)$ and solving, we get:\[ax+11=ax+a+x+1\]\[11=a+x+1\]\[a+x=10\qquad (3)\]

Multiplying $(2)$ by $(a+3)$ and solving, we get:\[ax+33=ax+2a+3x+6\]\[33=2a+3x+6\]\[2a+3x=27\qquad (4)\]

Solving the system of equations for $(3)$ and $(4)$, we find that $a=3$ and $x=\boxed{7}$.

Q6. Positive real numbers $x$ and $y$ satisfy $y^3=x^2$ and $(y-x)^2=4y^2$. What is $x+y$?
A6. Because $y^3=x^2$, set $x=a^3$, $y=a^2$ ($a\neq 0$). Put them in $(y-x)^2=4y^2$ we get $(a^2(a-1))^2=4a^4$ which implies $a^2-2a+1=4$. Solve the equation to get $a=3$ or $-1$. Since $x$ and $y$ are positive, $a=3$ and $x+y=3^3+3^2=\boxed{36}$.

Q7. What is the degree measure of the acute angle formed by lines with slopes $2$ and $\frac{1}{3}$?
A7. Reminder that $\text{slope}=\dfrac{\Delta y}{\Delta x}=\tan \theta$ where $\theta$ is the angle between the slope and $x$-axis. $k_1=2=\tan \alpha$, $k_2=\dfrac{1}{3}=\tan \beta$. The angle formed by the two lines is $\alpha-\beta$. $\tan(\alpha-\beta)=\dfrac{\tan\alpha-\tan\beta}{1+\tan\alpha\tan\beta}=\dfrac{2-1/3}{1+2\cdot 1/3}=1$. Therefore, $\alpha-\beta=\boxed{45}$.

Q8. In a table tennis tournament every participant played every other participant exactly once. Although there were twice as many right-handed players as left-handed players, the number of games won by left-handed players was $40\%$ more than the number of games won by right-handed players. (There were no ties and no ambidextrous players.) What is the total number of games played?
A8. We know that the total amount of games must be the sum of games won by left and right handed players. Then, we can write $g = l + r$, and since $l = 1.4r$, $g = 2.4r$. Given that $r$ and $g$ are both integers, $g/2.4$ also must be an integer. From here we can see that $g$ must be divisible by 12, leaving only answers B and D. Now we know the formula for how many games are played in this tournament is $n(n-1)/2$, the sum of the first $n-1$ triangular numbers. Now, setting 36 and 48 equal to the equation will show that two consecutive numbers must have a product of 72 or 96. Clearly, $72=8*9$, so the answer is $\boxed{36}$.

Q9. How many complex numbers satisfy the equation $z^5=\overline{z}$, where $\overline{z}$ is the conjugate of the complex number $z$?
A9. When $z^5=\overline{z}$, there are two conditions: either $z=0$ or $z\neq 0$. When $z\neq 0$, since $|z^5|=|\overline{z}|$, $|z|=1$. $z^5\cdot z=z^6=\overline{z}\cdot z=|z|^2=1$. Consider the $r(\cos \theta +i\sin \theta)$ form, when $z^6=1$, there are 6 different solutions for $z$. Therefore, the number of complex numbers satisfying $z^5=\bar{z}$ is $\boxed{7}$.

Q10. Consider the set of complex numbers $z$ satisfying $|1+z+z^{2}|=4$. The maximum value of the imaginary part of $z$ can be written in the form $\tfrac{\sqrt{m}}{n}$, where $m$ and $n$ are relatively prime positive integers. What is $m+n$?
A10. First, substitute in $z=a+bi$.

\[|1+(a+bi)+(a+bi)^2|=4\]\[|(1+a+a^2-b^2)+ (b+2ab)i|=4\]\[(1+a+a^2-b^2)^2+ (b+2ab)^2=16\]\[(1+a+a^2-b^2)^2+ b^2(1+4a+4a^2)=16\]
Let $p=b^2$ and $q=1+a+a^2$

\[(q-p)^2+ p(4q-3)=16\]\[p^2-2pq+q^2   + 4pq -3p=16\]
We are trying to maximize $b=\sqrt p$, so we'll turn the equation into a quadratic to solve for $p$ in terms of $q$.

\[p^2+(2q-3)p+(q^2-16)=0\]\[p=\frac{(-2q+3)\pm \sqrt{-12q+73}}{2}\]
We want to maximize $p$. Since $q$ is always negatively contributing to $p$'s value, we want to minimize $q$.

Due to the trivial inequality: $q=1+a+a^2=(a+\frac 12)^2+\frac{3}4 \geq \frac{3}4$

If we plug $q$'s minimum value in, we get that $p$'s maximum value is\[p=\frac{(-2(\frac 34)+3)+ \sqrt{-12(\frac 34)+73}}{2}=\frac{\frac 32+ 8}{2}=\frac{19}{4}\]
Then\[b=\frac{\sqrt{19}}{2}\]and\[m+n=\boxed{21}]

Q11. What is the product of all solutions to the equation\[\log_{7x}2023\cdot \log_{289x}2023=\log_{2023x}2023\]
A11. For $\log_{7x}2023\cdot \log_{289x}2023=\log_{2023x}2023$, transform it into $\dfrac{\ln 289+\ln 7}{\ln 7 + \ln x}\cdot \dfrac{\ln 289+\ln 7}{\ln 289 + \ln x}=\dfrac{\ln 289+\ln 7}{\ln 289+\ln 7+\ln x}$. Replace $\ln x$ with $y$. Because we want to find the product of all solutions of $x$, it is equivalent to finding the exponential of the sum of all solutions of $y$. Change the equation to standard quadratic equation form, the term with 1 power of $y$ is canceled. By using Vieta, we see that since there does not exist a $by$ term, $\sum y=0$ and $\prod x=e^0=\boxed{1}$.

Q12. Rows 1, 2, 3, 4, and 5 of a triangular array of integers are shown below.

[asy] size(4.5cm); label("$1$", (0,0)); label("$1$", (-0.5,-2/3)); label("$1$", (0.5,-2/3)); label("$1$", (-1,-4/3)); label("$3$", (0,-4/3)); label("$1$", (1,-4/3)); label("$1$", (-1.5,-2)); label("$5$", (-0.5,-2)); label("$5$", (0.5,-2)); label("$1$", (1.5,-2)); label("$1$", (-2,-8/3)); label("$7$", (-1,-8/3)); label("$11$", (0,-8/3)); label("$7$", (1,-8/3)); label("$1$", (2,-8/3)); [/asy]
Each row after the first row is formed by placing a 1 at each end of the row, and each interior entry is 1 greater than the sum of the two numbers diagonally above it in the previous row. What is the units digits of the sum of the 2023 numbers in the 2023rd row?
A12. First, let $R(n)$ be the sum of the $n$th row. Now, with some observation and math instinct, we can guess that $R(n) = 2^n - n$.

Now we try to prove it by induction,

$R(1) = 2^n - n = 2^1 - 1 = 1$ (works for base case)

$R(k) = 2^k - k$

$R(k+1) = 2^{k+1} - (k + 1) = 2(2^k) - k - 1$

By definition from the question, the next row is always$:$

Double the sum of last row (Imagine each number from last row branches off toward left and right to the next row), plus # of new row, minus 2 (minus leftmost and rightmost's 1)

Which gives us $:$

$2(2^k - k) + (k + 1) - 2 = 2(2^k) - k - 1$

Hence, proven

Last, simply substitute $n = 2023$, we get $R(2023) = 2^{2023} - 2023$

Last digit of $2^{2023}$ is $8$, $8-3 = \boxed{5}$

Q13. Let $f$ be the unique function defined on the positive integers such that\[\sum_{d\mid n}d\cdot f\left(\frac{n}{d}\right)=1\]for all positive integers $n$. What is $f(2023)$?
A13. From the problem, we want to find $f(2023)$. Using the problem, we get $f(2023)+7f(289)+17f(119)+119f(17)+289f(7)+2023f(1)=1$. By plugging in factors of $2023$, we get\begin{align} f(7)+7f(1)=1\\ f(17)+17f(1)=1\\ f(119)+7f(17)+17f(7)+119f(1)=1\\ f(289)+17f(17)+289f(1)=1 \end{align}Notice that $(4)-17(2)=f(289)$, so $f(289)=-16$. Similarly, notice that $(3)-17(1)=f(119)+7f(17)=-16$. Now, substituting this all back into our equation to solve for $f(2023)$, we get\begin{align*} f(2023)+7f(289)+17(f(119)+7f(17))+289(f(7)+7f(1))=1\\ f(2023)+7 \cdot (-16) + 17 \cdot (-16) + 289 \cdot (1) = 1\\ f(2023)=\boxed{96} \end{align*}

Q14. How many ordered pairs of positive real numbers $(a,b)$ satisfy the equation\[(1+2a)(2+2b)(2a+b) = 32ab?\]
A14. Using AM-GM on the two terms in each factor on the left, we get\[(1+2a)(2+2b)(2a+b) \ge 8\sqrt{2a \cdot 4b \cdot 2ab}= 32ab,\]meaning the equality condition must be satisfied. This means $1 = 2a = b$, so we only have $\boxed{1}$ solution.

Q15. Let $K$ be the number of sequences $A_1$, $A_2$, $\dots$, $A_n$ such that $n$ is a positive integer less than or equal to $10$, each $A_i$ is a subset of $\{1, 2, 3, \dots, 10\}$, and $A_{i-1}$ is a subset of $A_i$ for each $i$ between $2$ and $n$, inclusive. For example, $\{\}$, $\{5, 7\}$, $\{2, 5, 7\}$, $\{2, 5, 7\}$, $\{2, 5, 6, 7, 9\}$ is one such sequence, with $n = 5$.What is the remainder when $K$ is divided by $10$?
A15. Consider any sequence with $n$ terms. Every 10 number has such choices: never appear, appear the first time in the first spot, appear the first time in the second spot… and appear the first time in the $n$th spot, which means every number has $(n+1)$ choices to show up in the sequence. Consequently, for each sequence with length $n$, there are $(n+1)^{10}$ possible ways.

Thus, the desired value is $\sum_{i=1}^{10}(i+1)^{10}\equiv \boxed{5}\pmod{10}$

"""    
    
code = aops + """Below is another math problem you are to solve (non-negative integer):
\"{}\"
Determine a sympy-based approach for solving the problem by listing each step to take and what functions need to be called in each step. Remember, your final answer should be a non-negative integer, not an algebraic expression.
Write the entire script covering all the steps (use comments and document it well) and print the result. After solving the problem, output the final numerical answer within \\boxed{}.

Approach:"""


cot = """Below is a math problem you are to solve (non-negative integer):
\"{}\"
Analyze this problem and think step by step to come to a solution with programs. After solving the problem, output the final numerical answer within \\boxed{}.\n\n"""

promplt_options = [code,cot]
    
tool_instruction = '\n\nPlease integrate natural language reasoning with programs to solve the above problem, and put your final numerical answer within \\boxed{}.\nNote that the intermediary calculations may be real numbers, but the final numerical answer would always be an integer.'

#tool_instruction = " The answer should be given as a non-negative modulo 1000."
#tool_instruction += '\nPlease integrate natural language reasoning with programs to solve the problem above, and put your final answer within \\boxed{}.'

temperature = 0.9
top_p = 1.0

temperature_coding = 0.9
top_p_coding = 1.0

In [3]:
def get_package_version(package_name):
    try:
        version = pkg_resources.get_distribution(package_name).version
        return version
    except pkg_resources.DistributionNotFound:
        return "Package not found"

In [4]:
print('python版本：',sys.version)

package_name = "torch"
version = get_package_version(package_name)
print(f"{package_name}版本：{version}")

cuda_version = torch.version.cuda
print("CUDA版本：", cuda_version)

python版本： 3.10.13 | packaged by conda-forge | (main, Dec 23 2023, 15:36:39) [GCC 12.3.0]
torch版本：2.1.2
CUDA版本： 12.1


In [5]:
## Forked From  https://kaggle.com/code/xiaoz259/pure-rng/notebook


# credits:
# https://www.kaggle.com/code/olyatsimboy/aimo-openmath-mistral-baseline
# https://www.kaggle.com/code/aatiffraz/prompt-prediction-w-mixtral-mistral7b-gemma-llama
# https://www.kaggle.com/code/thedrcat/aimo-mixtral-baseline

In [6]:
NOTEBOOK_START_TIME = time.time()

In [7]:
env = aimo.make_env()
iter_test = env.iter_test()

TO-DO

Change temperature as the question goes longer
Change temperature based on question lenght

# Zero-shot MMOS-DeepSeekMath-7B with self-consistency and generated code reasoning evaluation

Self-consistency is a modification of the standard greedy decoding in reasoning pipelines via sampling several diverse answers followed by aggregation, e.g., most common answer ([SC-CoT paper](https://arxiv.org/pdf/2203.11171.pdf)).

In this kernel, we will consider MMOS-DeepSeekMath-7B RL-tuned backbone; in my experiments, this model produces more consistent code reasoning and the code block execution will allow us to decrease arithmetic hallucinations.

In [8]:
if QUANT:
    from transformers import BitsAndBytesConfig
    quantization_config = BitsAndBytesConfig(
        load_in_4bit = True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
    )

In [9]:
%%time
if QUANT:
    !pip install -U /kaggle/input/accelerate-wheelwhl/accelerate-0.29.1-py3-none-any.whl -qq
    !pip install -U /kaggle/input/bitsandbytes-0-42-0-py3-none-any-whl/bitsandbytes-0.42.0-py3-none-any.whl -qq

torch.backends.cuda.enable_mem_efficient_sdp(False)

print(f"Transformers Version: {transformers.__version__}")
set_seed(42)

Transformers Version: 4.39.3


2024-06-10 15:27:30.017283: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-10 15:27:30.017404: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-10 15:27:30.146255: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


CPU times: user 6.2 s, sys: 1.04 s, total: 7.24 s
Wall time: 10.6 s


In [10]:
# df = pd.read_csv('/kaggle/input/ai-mathematical-olympiad-prize/test.csv')
# df.head()

In [11]:
# if len(df) < 5:
#     df = pd.read_csv('/kaggle/input/ai-mathematical-olympiad-prize/train.csv')
#     PRIVATE = False
# df.head()

In [12]:
def naive_parse(answer):
    out = []
    start = False
    end = False
    for l in reversed(list(answer)):
        if l in '0123456789' and not end:
            start = True
            out.append(l)
        else:
            if start:
                end = True
        
    out = reversed(out)
    return ''.join(out)

In [13]:
def return_last_print(output, n):
    lines = output.strip().split('\n')
    if lines:
        return lines[n]
    else:
        return ""

def process_code(code, return_shell_output=False):
    
    def repl(match):
        if "real" not in match.group():
            return "{}{}".format(match.group()[:-1], ', real=True)')
        else:
            return "{}{}".format(match.group()[:-1], ')')
    code = re.sub(r"symbols\([^)]+\)", repl, code)

    if return_shell_output:
        code = code.replace('\n', '\n    ')
            # Add a try...except block
        code = "\ntry:\n    from sympy import *\n{}\nexcept Exception as e:\n    print(e)\n    print('FAIL')\n".format(code)
    
    if not return_shell_output:
        print(code)
    with open('code.py', 'w') as fout:
        fout.write(code)
    
    batcmd = 'timeout 7 ' + sys.executable + ' code.py'
    try:
        shell_output = subprocess.check_output(batcmd, shell=True).decode('utf8')
        return_value = return_last_print(shell_output, -1)
        print(shell_output)
        if return_shell_output:
            if return_value=='FAIL':
                CODE_STATUS = False
                return_value = return_last_print(shell_output, -2)
                if "not defined" in return_value:
                    return_value+='\nTry checking the formatting and imports'
            else:
                CODE_STATUS = True
            return return_value, CODE_STATUS  
        code_output = round(float(eval(return_value))) % 1000
    except Exception as e:
        print(e,'shell_output')
        code_output = -1
    
    if return_shell_output:
        if code_output==-1:
            CODE_STATUS = False
        else:
            CODE_STATUS = True
        return code_output, CODE_STATUS  
    
    
    return code_output


def process_text_output(output):
    result = output    
    try:
        result_output = re.findall(r'\\boxed\{(\d+)\}', result)

        print('BOXED', result_output)
        if not len(result_output):
            result_output = naive_parse(result)
        else:
            result_output = result_output[-1]

        print('BOXED FINAL', result_output)
        if not len(result_output):
            result_output = -1
        
        else:
            result_output = round(float(eval(result_output))) % 1000
    
    except Exception as e:
        print(e)
        print('ERROR PARSING TEXT')
        result_output = -1
    
    return result_output


In [14]:
torch.cuda.empty_cache()
gc.collect()

0

In [15]:
if PRIVATE:
    config = AutoConfig.from_pretrained(MODEL_PATH)
    config.gradient_checkpointing = True

    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

    device_map = [('model.embed_tokens', 0),
                 ('model.layers.0', 0),
                 ('model.layers.1', 0),
                 ('model.layers.2', 0),
                 ('model.layers.3', 0),
                 ('model.layers.4', 0),
                 ('model.layers.5', 0),
                 ('model.layers.6', 0),
                 ('model.layers.7', 0),
                 ('model.layers.8', 0),
                 ('model.layers.9', 0),
                 ('model.layers.10', 0),
                 ('model.layers.11', 0),
                 ('model.layers.12', 0),
                 ('model.layers.13', 0),
                 ('model.layers.14', 0),
                 ('model.layers.15', 0),
                 ('model.layers.16', 0),
                 ('model.layers.17', 0),
                 ('model.layers.18', 0),
                 ('model.layers.19', 0),
                 ('model.layers.20', 0),
                 ('model.layers.21', 0),
                 ('model.layers.22', 1),
                 ('model.layers.23', 1),
                 ('model.layers.24', 1),
                 ('model.layers.25', 1),
                 ('model.layers.26', 1),
                 ('model.layers.27', 1),
                 ('model.layers.28', 1),
                 ('model.layers.29', 1),
                 ('model.norm', 1),
                 ('lm_head', 1)]

    device_map = {ii:jj for (ii,jj) in device_map}

    if QUANT:
        from transformers import BitsAndBytesConfig
        quantization_config = BitsAndBytesConfig(
            load_in_4bit = True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16,
            bnb_4bit_use_double_quant=True,
        )
        model = AutoModelForCausalLM.from_pretrained(
            MODEL_PATH,
            device_map="sequential",
            torch_dtype="auto",
            trust_remote_code=True, 
            quantization_config=quantization_config,
            config=config
        )
    else:  
        model = AutoModelForCausalLM.from_pretrained(
            MODEL_PATH,
            device_map=device_map,
            torch_dtype="auto",
            trust_remote_code=True,
            #quantization_config=quantization_config,
            config=config
        )
    
    pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype='auto',
    device_map=device_map,
)

    class StoppingCriteriaSub(StoppingCriteria):
        def __init__(self, stops = [], encounters=1):
            super().__init__()
            self.stops = [stop.to("cuda") for stop in stops]

        def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
            for stop in self.stops:
                last_token = input_ids[0][-len(stop):]
                if torch.all(torch.eq(stop,last_token)):
                    return True
            return False


    stop_words = ["```output", "```python", "```\nOutput" , ")\n```" , "``````output"] #,  
    stop_words_ids = [tokenizer(stop_word, return_tensors='pt', add_special_tokens=False)['input_ids'].squeeze() for stop_word in stop_words]
    stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_words_ids)])
    
    model.dtype, model.hf_device_map

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [16]:
total_results = {}
total_answers = {}
best_stats = {}
total_outputs = {}
question_type_counts = {}
starting_counts = (2,3)

if VALIDATION:
    x = pd.read_csv(EXTERNAL_DF)
    y = x.copy()
    y['row_id'] = range(0, len(x))
    y['answer'] = 0
    y = y.drop(columns=['problem'])
    iterable = zip(x.iterrows(), y.iterrows())
else:
    iterable = iter_test
    
# LEWIS: I had to invert the loop order because the new API forbids repeated calls on the same problem
for i, (test, sample_submission) in tqdm(enumerate(iterable)):
    print(f"Solving problem {i} ...")
    TIME_SPENT = time.time() - NOTEBOOK_START_TIME

    if TIME_SPENT>TIME_LIMIT:
        sample_submission['answer'] = 0
        env.predict(sample_submission)
        break
        
    for jj in tqdm(range(n_repetitions)):   
#         id_ = df['id'].loc[i]
#         problem = df['problem'].loc[i]

        if VALIDATION:
            problem = test[1]['problem']
        else:
            problem = test['problem'].values[0]
        print(f"\n\n\nQUESTION {i} - {jj} - TIME_SPENT : {TIME_SPENT:.0f} secs")
        
        best, best_count = best_stats.get(i,(-1,-1))
        if best_count>np.sqrt(jj):
            print("SKIPPING CAUSE ALREADY FOUND BEST")
            continue
            
        outputs = total_outputs.get(i,[])
        text_answers, code_answers = question_type_counts.get(i,starting_counts)
        results = total_results.get(i,[])
        answers = total_answers.get(i,[])
        
        for _ in range(5):
            torch.cuda.empty_cache()
            gc.collect()
            time.sleep(0.2)

        try:
            ALREADY_GEN = 0
            code_error = None
            code_error_count = 0
            code_output = -1
            #initail_message = problem  + tool_instruction 
            counts = np.array([text_answers,code_answers])

            draw = choice(promplt_options, 1,
                          p=counts/counts.sum())

            initail_message = draw[0].format(problem,"{}")            
            prompt = f"User: {initail_message}"

            current_printed = len(prompt)
            print(f"{jj}_{prompt}\n")

            model_inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
            input_len = len(model_inputs['input_ids'][0])

            generation_output = model.generate(**model_inputs, 
                                               max_new_tokens=TOTAL_TOKENS-ALREADY_GEN,
                                               return_dict_in_generate=USE_PAST_KEY,
                                               do_sample = True,
                                               temperature = temperature,
                                               top_p = top_p,
                                               num_return_sequences=1, stopping_criteria = stopping_criteria)

            if USE_PAST_KEY:
                output_ids = generation_output.sequences[0]
            else:
                output_ids = generation_output[0]
            decoded_output = tokenizer.decode(output_ids, skip_special_tokens=True)
            print(f"{decoded_output[current_printed:]}\n")
            current_printed += len(decoded_output[current_printed:])
            cummulative_code = ""
            
            
            stop_word_cond = False
            for stop_word in stop_words:
                stop_word_cond = stop_word_cond or (decoded_output[-len(stop_word):]==stop_word)
                
            
            while (stop_word_cond) and (ALREADY_GEN<(TOTAL_TOKENS)):

                if (decoded_output[-len("```python"):]=="```python"):
                    temperature_inner=temperature_coding
                    top_p_inner = top_p_coding
                    prompt = decoded_output
                else:
                    temperature_inner=temperature
                    top_p_inner = top_p
                    try:
                        if (decoded_output[-len("``````output"):]=="``````output"):
                            code_text = decoded_output.split('```python')[-1].split("``````")[0]
                        else:
                            code_text = decoded_output.split('```python')[-1].split("```")[0]
                        

                        cummulative_code+=code_text
                        code_output, CODE_STATUS = process_code(cummulative_code, return_shell_output=True)
                        print('CODE RESULTS', code_output)

                        if code_error==code_output:
                            code_error_count+=1
                        else:
                            code_error=code_output
                            code_error_count = 0

                        if not CODE_STATUS:
                            cummulative_code = cummulative_code[:-len(code_text)]

                            if code_error_count>=1:
                                print("REPEATED ERRORS")
                                break

                    except Exception as e:
                        print(e)
                        print('ERROR PARSING CODE')
                        code_output = -1

                    if code_output!=-1:
                        if (decoded_output[-len(")\n```"):]==")\n```"):
                            prompt = decoded_output+'```output\n'+str(code_output)+'\n```\n'
                        else:
                            prompt = decoded_output+'\n'+str(code_output)+'\n```\n'
                    else:
                        prompt = decoded_output
                        cummulative_code=""


                model_inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
                ALREADY_GEN =  len(model_inputs['input_ids'][0])-input_len

                if USE_PAST_KEY:
                    old_values = generation_output.past_key_values
                else:
                    old_values = None

                generation_output = model.generate(**model_inputs, 
                                                   max_new_tokens=TOTAL_TOKENS-ALREADY_GEN, 
                                                   return_dict_in_generate=USE_PAST_KEY,
                                                   past_key_values=old_values,
                                                   do_sample = True,
                                                   temperature = temperature_inner,
                                                   top_p = top_p_inner,
                                                   num_return_sequences=1, stopping_criteria = stopping_criteria)

                if USE_PAST_KEY:
                    output_ids = generation_output.sequences[0]
                else:
                    output_ids = generation_output[0]
                decoded_output = tokenizer.decode(output_ids, skip_special_tokens=True)
                print(f"\nINTERMEDIATE OUT :\n{decoded_output[current_printed:]}\n")
                current_printed+=len(decoded_output[current_printed:])
                
                stop_word_cond = False
                for stop_word in stop_words:
                    stop_word_cond = stop_word_cond or (decoded_output[-len(stop_word):]==stop_word)

            if USE_PAST_KEY:
                output_ids = generation_output.sequences[0]
            else:
                output_ids = generation_output[0]

            raw_output = tokenizer.decode(output_ids[input_len:], skip_special_tokens=True)
            #print(f"\n\nOutput :\n{raw_output}\n")                            
            result_output = process_text_output(raw_output)
            
            try:
                code_output = round(float(eval(code_output))) % 1000
            except Exception as e:
                print(e,'final_eval')
                code_output = -1

        except Exception as e:
            print(e,"5")
            result_output, code_output = -1, -1

        if code_output!=-1:
            outputs.append(code_output)
            code_answers+=1

        if result_output!=-1:
            outputs.append(result_output)
            text_answers+=1

        if len(outputs) > 0:
            occurances = Counter(outputs).most_common()
            print(occurances)
            if occurances[0][1] > best_count:
                print("GOOD ANSWER UPDATED!")
                best = occurances[0][0]
                best_count = occurances[0][1]
            if occurances[0][1] > 5:
                print("ANSWER FOUND!")
                break

        results.append(result_output)
        answers.append(code_output)
        
        best_stats[i] = (best, best_count) 
        question_type_counts[i] = (text_answers, code_answers)
        total_outputs[i] = outputs
        
        total_results[i] = results
        total_answers[i] = answers

        print("code_answers",code_answers-starting_counts[1],"text_answers",text_answers-starting_counts[0])
        if DEBUG:
            break
            
    print(f"Predicted best answer: {best_stats}")
    if VALIDATION:
        y.loc[i, 'answer'] = best_stats[i][0]
    else:
        sample_submission['answer'] = best_stats[i][0]
        env.predict(sample_submission)

0it [00:00, ?it/s]

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
Solving problem 0 ...



  0%|          | 0/1 [00:00<?, ?it/s]




QUESTION 0 - 0 - TIME_SPENT : 159 secs



100%|██████████| 1/1 [00:02<00:00,  2.23s/it]
1it [00:02,  2.26s/it]

...

```python


INTERMEDIATE OUT :

from sympy import symbols, Eq, solve

def solve_equation():
    """Solve $4+x=4$ for $x$."""
    x = symbols('x')
    equation = Eq(4 + x, 4)
    solution = solve(equation, x)[0]

    return solution

result = solve_equation()
print(result)
```



Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


0

CODE RESULTS 0



100%|██████████| 1/1 [00:10<00:00, 10.35s/it]
3it [00:22,  7.55s/it]


INTERMEDIATE OUT :
```output
0
```
The solution for $x$ is $0$.
The answer is $\boxed{0}$.

BOXED ['0']
BOXED FINAL 0
[(0, 2)]
GOOD ANSWER UPDATED!
code_answers 1 text_answers 1
Predicted best answer: {0: (-1, -1), 1: (0, 2), 2: (0, 2)}


In [17]:
if VALIDATION:
    x['pred'] = y['answer']
    print('Correct Answers: {i}'.format(i = sum(x['pred'] == x['answer'])))
else:
    with open('code.py', 'w') as fout:
        fout.write("print('done')")

    batcmd = 'timeout 7 ' + sys.executable + ' code.py'
    try:
        shell_output = subprocess.check_output(batcmd, shell=True).decode('utf8')
        print(shell_output)
    except:
        pass

done

